In [28]:
import pymongo
import math 
# import re
import pandas as pd
# import numpy as np
import pylab 
from matplotlib import pyplot as plt
# from scipy import stats 
from  qgrid import show_grid as show 
from datetime import datetime
# import seaborn as sns

class testCase(object): 
    """in general its just the name of the machine where we launched the test """
    def __init__ (self , testname,database='rapls2',serveraddr='172.16.45.8',serverport=27017): 
        self._client = pymongo.MongoClient(serveraddr, serverport)
        self._db=self._client[database]
        self._testname=testname 
        self._sensors=self._db['sensor'+self._testname]
        
    def gettimetamps(self,containername): 
        containerdata=list(self._sensors.find({'target':containername},projection=['timestamp']))
        begintime= containerdata[0]['timestamp']
        endtime=containerdata[-1]['timestamp']
        return begintime , endtime 

    def getpowers(self,containername): 
        #get the power consumption of the system  between begin and end 
        begin , end = self.gettimetamps(containername) 
        x=list(self._sensors.find({'target':'system','timestamp' :{'$gte':begin,'$lte':end}},projection=['rapl','timestamp']))
        conso= pd.DataFrame(x)
        sonde=next(iter(x[0]['rapl']['0']))
        conso['power']=conso['rapl'].apply(lambda row :math.ldexp( row['0'][sonde]['RAPL_ENERGY_PKG'],-32))
        return conso.loc[:,['timestamp','power']]

    def getenergy(self,containername):
        powers =self.getpowers(containername)
        return powers['power'].sum()

    def getrecap(self): 
        meausres = self._db['recap'+self._testname].find(projection={'_id': False,'id':False})
        return pd.DataFrame(list(meausres))
    




# simple example

In [29]:
#getting an instance of our machine 
machine= testCase('tt1230119')


In [30]:
#to print all the meausres done in this machine 
machine.getrecap()
## this only used if you launched the test with tester.sh 

,execution energy,execution time,name,warmup energy,warmup time
0,300.015381,12,yolo,0.0,0
